# Change instruments feature

In [1]:
import os
from copy import copy
import ms3
from bs4 import BeautifulSoup

This is some code I wrote recently to do one particular instrument change on a score. The use case was that I had a bunch of scores that had one **piano** staff but the music it contained was actually for **drumset**. So I had to change the instrument and also systematically replace all notes. This latter part I removed from the notebook because it's not relevant but the `repair_part()` function might be a good starting point for your task. I developed it by doing the instrument change manually in MuseScore, inspecting the changes in the `.mscx` file, and then simply performing the required modifications on the XML structure. I've added a couple of comments to remind you the BeautifulSoup interface. In the comments, I'm using the words "tag" and "node" interchangeably.

In [7]:
def append_new_tag(soup, append_to_tag, new_tag_name, new_tag_value=None, **attributes):
    new_tag = soup.new_tag(new_tag_name, **attributes)
    if new_tag_value is not None:
        new_tag.string = new_tag_value
    append_to_tag.append(new_tag)
    
def replace_or_create(soup, parent_tag, tag_to_change, new_value):
    modify_this = parent_tag.find(tag_to_change)
    if modify_this is not None:
        modify_this.string = new_value
    else:
        append_new_tag(soup, parent_tag, tag_to_change, new_value)
        

def repair_part(soup: BeautifulSoup):
    """Applies the specific changes when changing an instrument to drumset."""
    part_tag = soup.find("Part") # get the first <Part> tag of the whole tree, no matter where it is
    staff_tag = part_tag.Staff # direct access to the <Staff> child node not of the <Part> tag
    st_type_tag = staff_tag.StaffType 
    st_type_tag["group"] = "percussion" # changes the "group" attribute to <StaffType group="percussion">
    st_type_tag.find("name").string = "perc5Line" # get the first <name> tag within the <StaffType> subtree
    append_new_tag(soup, st_type_tag, "keysig", "PERC")
    instrument_tag = part_tag.Instrument
    instrument_tag["id"] = "drumset"
    replace_or_create(soup, instrument_tag, "longName", "Drumset")
    replace_or_create(soup, instrument_tag, "shortName", "D. Set")
    replace_or_create(soup, instrument_tag, "trackName", "Drumset")
    replace_or_create(soup, instrument_tag, "instrumentId", "drum.group.set")
    append_new_tag(soup, instrument_tag, "useDrumset", "1")
    append_new_tag(soup, instrument_tag, "clef", "PERC")
    channel_tag = instrument_tag.Channel
    for controller_tag in channel_tag.find_all("controller"):
        controller_tag.decompose() # removes all existing <controller> child
    append_new_tag(soup, channel_tag, "controller", ctrl="0", value="1")
    append_new_tag(soup, channel_tag, "controller", ctrl="32", value="0")
    
def store_modified(score, suffix=''):
    file_name = score.fnames['mscx'] + f'{suffix}.mscx'
    file_path = os.path.join(score.paths['mscx'], file_name)
    score.store_score(file_path)
    print(f"Written {file_path}")
    
def repair_score(file_path, suffix='', verbose=False):
    score = ms3.Score(file_path)
    soup = score.mscx.parsed.soup
    repair_part(soup)
    store_modified(score, suffix)

In [19]:
path = "/home/hentsche/unittest_metacorpus/mixed_files/changed_instruments/Brahms Op. 99iv.mscx"
repair_score(path, suffix='')

Written /home/hentsche/unittest_metacorpus/mixed_files/changed_instruments/Brahms Op. 99iv.mscx


In [21]:
score = ms3.Score(path)
score.mscx.metadata['parts']

{'part_1': {'staves': [1],
  'trackName': 'Cello',
  'longName': 'Drumset',
  'shortName': 'D. Set',
  'instrument': 'Drumset',
  'staff_1_ambitus': {'min_midi': 36,
   'min_name': 'C2',
   'max_midi': 72,
   'max_name': 'C5'}},
 'part_2': {'staves': [2, 3],
  'trackName': 'Piano',
  'longName': 'Piano',
  'shortName': 'Pno.',
  'instrument': 'Piano',
  'staff_2_ambitus': {'min_midi': 36,
   'min_name': 'C2',
   'max_midi': 96,
   'max_name': 'C7'},
  'staff_3_ambitus': {'min_midi': 24,
   'min_name': 'C1',
   'max_midi': 72,
   'max_name': 'C5'}}}

Since all the scores modified with this script had only one single staff, the code did not need to discriminate between the tags pertaining to different staves. If you open the modified Brahms score, for example, you will see that the only the upper staff (previously Cello) is changed to drumset.

# Your task

For development you can copy and adapt the `repair_score()` function to load, modify, and write back an individual score. This is where you can initialize the object from the new `Instrumentation(LoggedClass)` class that you will develop to conveniently change instruments. It will be initialized with `score.mscx.parsed.soup` (the XML tree) as the only argument. In the following, the variable `instrumentation` designates such an object instantiated as `Instrumentation(score.mscx.parsed.soup)`.

## Accessing instrument information

**`instrumentation[i]` (where i designates the ID of a staff) should return the currently set instrument name, identical to the value in the column `staff_<i>_instrument` for the respective column in `metadata.tsv`.** 

The `metadata['parts']` shown above is created by iterating through the soup's `<Part>` tags and getting the relevant information by calling `ms3.bs4_parser.get_part_info()`. Please study this function since it reveals the relevant XML structure. Your object will basically have to do the same thing, but with two important differences: (1) It will keep track of the references to the tags to be able to modify them when necessary. (2) It also needs to include the `<instrumentId>` tag. 

Every `<Part>` tag contains at least one `<Staff id="<i>">` tag and exactly one `<Instrument>` tag. In other words, the `<Part>` tag assigns the same `<Instrument>` to all contained `<Staff>` tags. Therefore, the `Instrumentation` object needs to maintain a mapping from staff IDs `i` to `<Part>` tags and yield the instrument of the corresponding part.

## Changing instrument

**`instrumentation[i] = 'InstrumentName'` (where i designates the ID of a staff) should change the instrument information for the respective `<Part>`.** 

Changing the value for one staff therefore changes it for all staves that are contained in the same `<Part>`. In the example dictionary above, `part_2` pertains to staves 2 & 3, left and right hand of the piano part. Therefore, setting `instrumentation[3] = 'Violin'` and then calling `instrumentation[2]` will yield `'Violin'`, too, and setting `instrumentation[2] = 'Violin'` will do nothing.

In [14]:
MIXED_FILES_FOLDER = "/home/hentsche/unittest_metacorpus/mixed_files/" 

target_files_folder = os.path.join(MIXED_FILES_FOLDER, "changed_instruments")
source_file2path = {}
for path, subdirs, files in os.walk(MIXED_FILES_FOLDER):
    current_folder = os.path.basename(path)
    if current_folder.startswith('.') or current_folder == "changed_instruments":
        continue
    musescore_files = [f for f in files if f.endswith('.mscx')]
    for file in musescore_files:
        source_path = os.path.join(path, file)
        source_file2path[file] = source_path
source_file2path

{'stabat_03_coloured.mscx': '/home/hentsche/unittest_metacorpus/mixed_files/stabat_03_coloured.mscx',
 '76CASM34A33UM.mscx': '/home/hentsche/unittest_metacorpus/mixed_files/76CASM34A33UM.mscx',
 'Brahms Op. 99iv.mscx': '/home/hentsche/unittest_metacorpus/mixed_files/chamber/Brahms Op. 99iv.mscx',
 '05_symph_fant.mscx': '/home/hentsche/unittest_metacorpus/mixed_files/orchestral/05_symph_fant.mscx',
 'Did03M-Son_regina-1762-Sarti.mscx': '/home/hentsche/unittest_metacorpus/mixed_files/orchestral/Did03M-Son_regina-1762-Sarti.mscx',
 'caldara_form.mscx': '/home/hentsche/unittest_metacorpus/mixed_files/orchestral/caldara_form.mscx',
 'BWV_0815.mscx': '/home/hentsche/unittest_metacorpus/mixed_files/keyboard/baroque/BWV_0815.mscx',
 '12.16_Toccata_cromaticha_per_l’elevatione_phrygian.mscx': '/home/hentsche/unittest_metacorpus/mixed_files/keyboard/ancient/12.16_Toccata_cromaticha_per_l’elevatione_phrygian.mscx',
 'Tempest_1st.mscx': '/home/hentsche/unittest_metacorpus/mixed_files/keyboard/ninet

### Test cases

Three cases introduced in https://github.com/DCMLab/unittest_metacorpus/compare/0a99b04...instruments

In [ ]:
[
    ('BWV_0815.mscx', {1: 'Harpsichord', 2: 'Harpsichord'}),
    ('Brahms Op. 99iv.mscx', {1: 'Piano', 2: 'Piano'}),
]